# Resample dff data to have the same speed distribution

In [ ]:
import numpy as np
import os
import pandas as pd
import glob

In [ ]:
# Define the directory path
directory_path = r'D:\2P_DATA\ChAT_MEC_switch_analysis_240822\Speed_min\Switch_timing'

# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

# Load each CSV file into a DataFrame and store in a list
switch = [pd.read_csv(file) for file in csv_files]

print(switch)

In [ ]:
# Define the directory path
directory_path = r'D:\2P_DATA\ChAT_MEC_switch_analysis_240822\Speed_min\df_downsample'

# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

# Load each CSV file into a DataFrame and store in a list
df_downsample = [pd.read_csv(file) for file in csv_files]

print(df_downsample)


In [ ]:
import numpy as np
import os

# Define the directory containing the .npy files
directory = r"D:\2P_DATA\ChAT_MEC_switch_analysis_240822\Novelty_sec\dff_f_man_from_each_dff"

# List all .npy files in the directory
file_list = [file for file in os.listdir(directory) if file.endswith('.npy')]

# Load all .npy files into a dictionary with filenames as keys
data_dict = {file: np.load(os.path.join(directory, file)) for file in file_list}

# Now, data_dict contains all the loaded arrays with filenames as keys
for filename, data in data_dict.items():
    print(f"Filename: {filename}")
    print(data)

In [ ]:
# Convert the arrays in the dictionary into a list of numpy arrays
dff_f_mean = [data for data in data_dict.values()]

# Optionally, you can inspect the result
print(dff_f_mean)
print("Number of arrays:", len(dff_f_mean))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from matplotlib import mlab
from scipy.signal import detrend

In [ ]:
# helper funtion
def bwlabel(data):
    count=0
    label= np.zeros((len(data)))
    if data[0]!=0:
        label[0]=1

    for i in range(1,len(data)):
        if (data[i]!=0) & (data[i-1]!=0):
            label[i]=count
        elif (data[i]!=0) & (data[i-1]==0):
            count=count+1
            label[i]=count

    return label

def double_thresh(data, lower_thresh,upper_thresh):
    state = np.zeros((len(data)))
    if data[0]<lower_thresh:
        state[0]=0
    else:
        state[0]=1
    
    for i in range(1,len(data)):
        state[i]=state[i-1]
        if state[i-1]==0:
            if data[i]>upper_thresh:
                state[i]=1
        else:
            if data[i]<lower_thresh:
                state[i]=0
    
    return state 

In [ ]:
# filter data by speed

def filter_running(speed, lower_thresh, upper_thresh, sit_thresh, run_thresh, max_value):
    """
    Filters running data based on thresholds for speed, immobility, and run duration.

    Args:
    - speed: Speed data (array).
    - lower_thresh: Lower threshold for running detection.
    - upper_thresh: Upper threshold for running detection.
    - sit_thresh: Minimum time (seconds) for immobility to be excluded.
    - run_thresh: Minimum time (seconds) for running to be included.
    - max_value: (Optional) Exclude data if speed exceeds this value.

    Returns:
    - running_index: Indices of data points considered as running.
    """
    # Double threshold
    run_state = double_thresh(speed, lower_thresh, upper_thresh)
    
    # Include small periods of immobility if smaller than sit_thresh seconds
    run_state_invert = np.where(run_state == 0, 1, 0)
    run_state_invert2 = bwlabel(run_state_invert)
    for i in range(int(np.max(run_state_invert2))):
        if len(run_state_invert2[run_state_invert2 == i]) < 60 * sit_thresh:
            run_state = np.where(run_state_invert2 == i, 1, run_state)
    
    # Exclude small periods of running if smaller than run_thresh seconds
    run_state2 = bwlabel(run_state)
    for i in range(int(np.max(run_state2))):
        if len(run_state2[run_state2 == i]) < 60 * run_thresh:
            run_state = np.where(run_state2 == i, 0, run_state)
    
    # Get running indices
    running_index = np.squeeze(np.array([ind for (ind, val) in np.ndenumerate(run_state) if val == 1]))

    # Exclude data points where speed exceeds the max_value, if provided
    if max_value is not None:
        running_index = np.array([idx for idx in running_index if speed[idx] <= max_value])
    
    return running_index


In [ ]:
speed_familiar_all=[] #velocity value
filtered_speed_minute_familiar=[] #velocity value
filtered_dff_minute_novel=[] #velocity value
filtered_dff_minute_Rfamiliar=[] #velocity value
filtered_dff_minute_familiar=[] #dff value
filtered_dff_minute_novel2=[] #dff value
filtered_dff_minute_Rfamiliar2=[] #dff value
running_index_familiar_list=[] #index
running_index_novel_list=[] #index
running_index_Rfamiliar_list=[] #index

for i in range(0,len(df_downsample)):
    velocity_f = savgol_filter(df_downsample[i].Velocity,50,3)
    velocity_f=pd.DataFrame(velocity_f)
    velocity_f_positive=velocity_f[0] #velocity_f[0]-velocity_f[0].min()+0.0000001
    switch1=switch[i].switch1.iloc[0]
    switch2=switch[i].switch2.iloc[0]
    # Filter running speed
    speed_familiar= np.array(velocity_f_positive[:switch1])
    running_index_familiar = filter_running(speed_familiar,lower_thresh=1.0,upper_thresh=1.0,sit_thresh=1,run_thresh=1,max_value=2) 
    speed_familiar = speed_familiar[running_index_familiar] 
    speed_novel= np.array(velocity_f_positive[switch1:switch2])#could use"int(switch1+(switch2-switch1)/2)" insteatd of "switch2"
    running_index_novel = filter_running(speed_novel,lower_thresh=0,upper_thresh=0,sit_thresh=1,run_thresh=1,max_value=6)
    speed_novel = speed_novel[running_index_novel]
    speed_Rfamiliar= np.array(velocity_f_positive[switch2:])
    running_index_Rfamiliar = filter_running(speed_Rfamiliar,lower_thresh=0,upper_thresh=0,sit_thresh=1,run_thresh=1,max_value=6)
    speed_Rfamiliar = speed_Rfamiliar[running_index_Rfamiliar] 
    speed_familiar_all=speed_familiar_all+[speed_familiar]
    running_index_familiar_list.append(running_index_familiar) #index
    running_index_novel_list.append(running_index_novel) #index
    running_index_Rfamiliar_list.append(running_index_Rfamiliar) #index
    
    df0=df_downsample[i][:switch1]
    df0=df0.reset_index().drop(columns="index")
    df0.Time=df0.Time-df0.Time[0]
    C_indices=running_index_familiar
    filtered_dff_average_minute_familiar_each=[]
    velocity_minute_each=[]
    for n in range(1,10+1):
        if n == 1:
            # Compute the absolute difference between the 'Time' column and 1
            abs_diff = np.abs(df0.Time[C_indices] - n*60)
            # Find the index of the minimum value in the absolute difference series
            X = np.argmin(abs_diff)
            filtered_dff_minute=dff_f_mean[i][:switch1][C_indices][:X]
            filtered_dff_average_minute_familiar_each.append(filtered_dff_minute)
            velocity_minute=velocity_f_positive[:switch1][C_indices][:X]
            velocity_minute_each.append(velocity_minute)
        else:
            abs_diff1 = np.abs(df0.Time[C_indices] - (n-1)*60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(df0.Time[C_indices] - n*60)
            X2 = np.argmin(abs_diff2)
            filtered_dff_minute=dff_f_mean[i][:switch1][C_indices][X1:X2]
            filtered_dff_average_minute_familiar_each.append(filtered_dff_minute)
            velocity_minute=velocity_f_positive[:switch1][C_indices][X1:X2]
            velocity_minute_each.append(velocity_minute)
    filtered_dff_minute_familiar += [filtered_dff_average_minute_familiar_each]
    filtered_speed_minute_familiar += [velocity_minute_each]
    
    df1=df_downsample[i][switch1:switch2]
    df1=df1.reset_index().drop(columns="index")
    df1.Time=df1.Time-df1.Time[0]
    #velocity_f = savgol_filter(df_downsample[i].Velocity,50,3)
    #velocity_f=pd.DataFrame(velocity_f)
    #velocity_f_positive=velocity_f[0]-velocity_f[0].min()+0.0000001
    velocity_f_positive_novel=velocity_f_positive[switch1:switch2]
    velocity_f_positive_novel=velocity_f_positive_novel.reset_index().drop(columns="index")
    velocity_f_positive_novel=np.array(velocity_f_positive_novel).ravel()
    C_indices=running_index_novel
    filtered_dff_average_minute_novel_each=[]
    filtered_dff_average_minute_novel_each2=[]
    for n in range(1,20+1):
        if n == 1:
            # Compute the absolute difference between the 'Time' column and 1
            abs_diff = np.abs(df1.Time[C_indices] - n*60)
            # Find the index of the minimum value in the absolute difference series
            X = np.argmin(abs_diff)
            filtered_dff_minute=velocity_f_positive_novel[C_indices][:X]
            filtered_dff_average_minute_novel_each.append(filtered_dff_minute) 
            filtered_dff_minute2=dff_f_mean[i][switch1:switch2][C_indices][:X]
            filtered_dff_average_minute_novel_each2.append(filtered_dff_minute2) 
        else:
            abs_diff1 = np.abs(df1.Time[C_indices] - (n-1)*60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(df1.Time[C_indices] - n*60)
            X2 = np.argmin(abs_diff2)
            filtered_dff_minute=velocity_f_positive_novel[C_indices][X1:X2]
            filtered_dff_average_minute_novel_each.append(filtered_dff_minute) 
            filtered_dff_minute2=dff_f_mean[i][switch1:switch2][C_indices][X1:X2]
            filtered_dff_average_minute_novel_each2.append(filtered_dff_minute2) 
    filtered_dff_minute_novel += [filtered_dff_average_minute_novel_each]
    filtered_dff_minute_novel2 += [filtered_dff_average_minute_novel_each2]
    
    df2=df_downsample[i][switch2:]
    df2=df2.reset_index().drop(columns="index")
    df2.Time=df2.Time-df2.Time[0]
    velocity_f_positive_Rfamiliar=velocity_f_positive[switch2:]
    velocity_f_positive_Rfamiliar=velocity_f_positive_Rfamiliar.reset_index().drop(columns="index")
    velocity_f_positive_Rfamiliar=np.array(velocity_f_positive_Rfamiliar).ravel()
    C_indices=running_index_Rfamiliar
    filtered_dff_average_minute_Rfamiliar_each=[]
    filtered_dff_average_minute_Rfamiliar_each2=[]
    for n in range(1,10+1):
        if n == 1:
            # Compute the absolute difference between the 'Time' column and 1
            abs_diff = np.abs(df2.Time[C_indices] - n*60)
            # Find the index of the minimum value in the absolute difference series
            X = np.argmin(abs_diff)
            filtered_dff_minute=velocity_f_positive_Rfamiliar[C_indices][:X]
            filtered_dff_average_minute_Rfamiliar_each.append(filtered_dff_minute) 
            filtered_dff_minute2=dff_f_mean[i][switch2:][C_indices][:X]
            filtered_dff_average_minute_Rfamiliar_each2.append(filtered_dff_minute2) 
        else:
            abs_diff1 = np.abs(df2.Time[C_indices] - (n-1)*60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(df2.Time[C_indices] - n*60)
            X2 = np.argmin(abs_diff2)
            filtered_dff_minute=velocity_f_positive_Rfamiliar[C_indices][X1:X2]
            filtered_dff_average_minute_Rfamiliar_each.append(filtered_dff_minute)
            filtered_dff_minute2=dff_f_mean[i][switch2:][C_indices][X1:X2]
            filtered_dff_average_minute_Rfamiliar_each2.append(filtered_dff_minute2) 
    filtered_dff_minute_Rfamiliar += [filtered_dff_average_minute_Rfamiliar_each]
    filtered_dff_minute_Rfamiliar2 += [filtered_dff_average_minute_Rfamiliar_each2]

In [ ]:
print(len(speed_familiar_all),len(filtered_dff_minute_novel),len(filtered_dff_minute_Rfamiliar)) 
print(len(filtered_dff_minute_familiar), len(filtered_dff_minute_novel2), len(filtered_dff_minute_Rfamiliar2))

In [ ]:
len(filtered_dff_minute_novel[0])

# Resampling dff each min

In [ ]:
#def distance(a, b):
    #return np.abs(a - b)

def distance(a, b):
    return (a - b) ** 2  # Squared difference for 1D

def resample_weighted(A, B, N):
    unique_A, counts_A = np.unique(A, return_counts=True)
    probabilities = counts_A / np.sum(counts_A)  # Probability of selecting each unique element in A
    
    newsample = np.zeros(N)  # Resampled points
    indices = np.zeros(N, dtype=int)  # Indices of the original B
    used_indices = set()  # To track which indices in B have been used
    
    for i in range(N):
        # Step 1: Weighted sampling based on the distribution of A
        a = np.random.choice(unique_A, p=probabilities)
        
        # Step 2: Find the closest point in B that hasn't been used
        mindist = np.inf
        mindistind = None
        for j in range(len(B)):
            if j not in used_indices:  # Skip already used indices
                d = distance(a, B[j])
                if d < mindist:
                    mindist = d
                    mindistind = j
        
        if mindistind is not None:
            newsample[i] = float(B[mindistind])  # Ensure it's a scalar
            indices[i] = mindistind
            used_indices.add(mindistind)  # Mark this index as used
        else:
            newsample[i] = np.nan  # Handle the case where no valid index was found

    # Sort the newsample based on indices to align with the original B order
    sorted_indices = np.argsort(indices)
    aligned_newsample = newsample[sorted_indices]
    aligned_indices = indices[sorted_indices]
    
    return aligned_newsample, aligned_indices


In [ ]:
#resample data in env2

from scipy.stats import mannwhitneyu

resampled_filtered_novel_all = []
resampled_filtered_novel_indices_all = []

for i in range(len(df_downsample)):
    resampled_mouse = []
    resampled_mouse_indices = []
    
    for n in range(20):
        A = speed_familiar_all[i]
        B = np.array(filtered_dff_minute_novel[i][n])
        # Ensure B is flattened to a 1D array before resampling
        #B = B.flatten()
        
        # Initialize resampling attempt counter
        attempt = 0
        p = 0  # Start with p > 0.05 to enter the loop
        resampled_filtered_novel = None  # Reset
        resampled_filtered_novel_indices = None  # Reset

        # Repeat resampling up to 10 times if p < 0.05
        while p <= 0.05 and attempt < 1000:
            resampled_filtered_novel, resampled_filtered_novel_indices = resample_weighted(A, B, 100)
            stat, p = mannwhitneyu(A, resampled_filtered_novel, alternative='two-sided')
            attempt += 1

        # If p is still < 0.05 after 10 attempts, set results to 0 and break
        if p <= 0.05 and attempt == 1000:
            resampled_filtered_novel = np.zeros(50)  # Zero array for consistency
            resampled_filtered_novel_indices = np.zeros(50)
            print(f"p-value still < 0.05 after 1000 attempts for n={n}, i={i}")

        # Append the results for the current n and i, ensure copying of valid data
        resampled_mouse += [np.copy(resampled_filtered_novel)]
        resampled_mouse_indices += [np.copy(resampled_filtered_novel_indices)]
    
    # Append results for each iteration of i
    resampled_filtered_novel_all += [resampled_mouse]
    resampled_filtered_novel_indices_all += [resampled_mouse_indices]

# Final output
print(f"Total length of resampled_filtered_novel_all: {len(resampled_filtered_novel_all)}")


In [ ]:
len(resampled_filtered_novel_all)

In [ ]:
filtered_resampled_dff_minute_novel = []

for i in range(len(df_downsample)):
    C_indices = resampled_filtered_novel_indices_all[i]
    filtered_resampled_dff_minute_novel_each = []

    for n in range(20):
        # Check if all values in C_indices[n] are 0
        if np.all(C_indices[n] == 0):
            filtered_dff_minute = None
        else:
            filtered_dff_minute = filtered_dff_minute_novel2[i][n][C_indices[n]].mean()
        
        # Append the result (None or calculated mean)
        filtered_resampled_dff_minute_novel_each.append(filtered_dff_minute)

    # Add the result for each 'i' iteration
    filtered_resampled_dff_minute_novel.append(filtered_resampled_dff_minute_novel_each)


In [ ]:
filtered_resampled_dff_minute_novel

In [ ]:
filtered_resampled_dff_minute_novel_all = []

for i in range(len(df_downsample)):
    C_indices = resampled_filtered_novel_indices_all[i]
    filtered_resampled_dff_minute_novel_all_each = []

    for n in range(20):
        # Check if all values in C_indices[n] are 0
        if np.all(C_indices[n] == 0):
            filtered_dff_minute = None
        else:
            filtered_dff_minute = filtered_dff_minute_novel2[i][n][C_indices[n]]
        
        # Append the result (None or calculated mean)
        filtered_resampled_dff_minute_novel_all_each.append(filtered_dff_minute)

    # Add the result for each 'i' iteration
    filtered_resampled_dff_minute_novel_all.append(filtered_resampled_dff_minute_novel_all_each)

In [ ]:
filtered_resampled_velocity_novel = []

for i in range(len(df_downsample)):
    C_indices = resampled_filtered_novel_indices_all[i]
    filtered_resampled_velocity_novel_each = []

    for n in range(20):
        # Check if all values in C_indices[n] are 0
        if np.all(C_indices[n] == 0):
            filtered_dff_minute = None
        else:
            filtered_dff_minute = filtered_dff_minute_novel[i][n][C_indices[n]]
        
        # Append the result (None or calculated mean)
        filtered_resampled_velocity_novel_each.append(filtered_dff_minute)

    # Add the result for each 'i' iteration
    filtered_resampled_velocity_novel.append(filtered_resampled_velocity_novel_each)

In [ ]:
#resample for returned env1 data

from scipy.stats import mannwhitneyu

resampled_filtered_Rfamiliar_all = []
resampled_filtered_Rfamiliar_indices_all = []

for i in range(len(df_downsample)):
    resampled_mouse = []
    resampled_mouse_indices = []
    
    for n in range(10):
        A = speed_familiar_all[i]
        B = np.array(filtered_dff_minute_Rfamiliar[i][n])
        # Ensure B is flattened to a 1D array before resampling
        #B = B.flatten()
        
        # Initialize resampling attempt counter
        attempt = 0
        p = 0  # Start with p > 0.05 to enter the loop
        resampled_filtered_Rfamiliar = None  # Reset
        resampled_filtered_Rfamiliar_indices = None  # Reset

        # Repeat resampling up to 10 times if p < 0.05
        while p <= 0.05 and attempt < 1000:
            resampled_filtered_Rfamiliar, resampled_filtered_Rfamiliar_indices = resample_weighted(A, B, 100)
            stat, p = mannwhitneyu(A, resampled_filtered_Rfamiliar, alternative='two-sided')
            attempt += 1

        # If p is still < 0.05 after 10 attempts, set results to 0 and break
        if p <= 0.05 and attempt == 1000:
            resampled_filtered_Rfamiliar = np.zeros(50)  # Zero array for consistency
            resampled_filtered_Rfamiliar_indices = np.zeros(50)
            print(f"p-value still < 0.05 after 1000 attempts for n={n}, i={i}")

        # Append the results for the current n and i, ensure copying of valid data
        resampled_mouse += [np.copy(resampled_filtered_Rfamiliar)]
        resampled_mouse_indices += [np.copy(resampled_filtered_Rfamiliar_indices)]
    
    # Append results for each iteration of i
    resampled_filtered_Rfamiliar_all += [resampled_mouse]
    resampled_filtered_Rfamiliar_indices_all += [resampled_mouse_indices]

# Final output
print(f"Total length of resampled_filtered_Rfamiliar_all: {len(resampled_filtered_Rfamiliar_all)}")


In [ ]:
filtered_resampled_dff_minute_Rfamiliar = []

for i in range(len(df_downsample)):
    C_indices = resampled_filtered_Rfamiliar_indices_all[i]
    filtered_resampled_dff_minute_Rfamiliar_each = []

    for n in range(10):
        # Check if all values in C_indices[n] are 0
        if np.all(C_indices[n] == 0):
            filtered_dff_minute = None
        else:
            filtered_dff_minute = filtered_dff_minute_Rfamiliar2[i][n][C_indices[n]].mean()
        
        # Append the result (None or calculated mean)
        filtered_resampled_dff_minute_Rfamiliar_each.append(filtered_dff_minute)

    # Add the result for each 'i' iteration
    filtered_resampled_dff_minute_Rfamiliar.append(filtered_resampled_dff_minute_Rfamiliar_each)


In [ ]:
filtered_resampled_dff_minute_Rfamiliar

In [ ]:
filtered_resampled_dff_minute_Rfamiliar_all = []

for i in range(len(df_downsample)):
    C_indices = resampled_filtered_Rfamiliar_indices_all[i]
    filtered_resampled_dff_minute_Rfamiliar_all_each = []

    for n in range(10):
        # Check if all values in C_indices[n] are 0
        if np.all(C_indices[n] == 0):
            filtered_dff_minute = None
        else:
            filtered_dff_minute = filtered_dff_minute_Rfamiliar2[i][n][C_indices[n]]
        
        # Append the result (None or calculated mean)
        filtered_resampled_dff_minute_Rfamiliar_all_each.append(filtered_dff_minute)

    # Add the result for each 'i' iteration
    filtered_resampled_dff_minute_Rfamiliar_all.append(filtered_resampled_dff_minute_Rfamiliar_all_each)


In [ ]:
filtered_resampled_velocity_Rfamiliar = []

for i in range(len(df_downsample)):
    C_indices = resampled_filtered_Rfamiliar_indices_all[i]
    filtered_resampled_velocity_Rfamiliar_each = []

    for n in range(10):
        # Check if all values in C_indices[n] are 0
        if np.all(C_indices[n] == 0):
            filtered_dff_minute = None
        else:
            filtered_dff_minute = filtered_dff_minute_Rfamiliar[i][n][C_indices[n]]
        
        # Append the result (None or calculated mean)
        filtered_resampled_velocity_Rfamiliar_each.append(filtered_dff_minute)

    # Add the result for each 'i' iteration
    filtered_resampled_velocity_Rfamiliar.append(filtered_resampled_velocity_Rfamiliar_each)

In [ ]:
filtered_dff_minute_ave_familiar=[]
for i in range(len(df_downsample)):
    filtered_dff_minute_ave_familiar_each=[]
    for n in range(10):
        filtered_dff_minute_ave_familiar_each.append(filtered_dff_minute_familiar[i][n].mean())
    filtered_dff_minute_ave_familiar+=[filtered_dff_minute_ave_familiar_each]
len(filtered_dff_minute_ave_familiar)

In [ ]:
import pickle

# Dictionary mapping file names to data variables
data_to_save = {
    "filtered_dff_minute_familiar": filtered_dff_minute_familiar,#dff data
    "filtered_dff_minute_ave_familiar": filtered_dff_minute_ave_familiar,#dff data
    "filtered_dff_minute_novel": filtered_dff_minute_novel,#velocity data
    "filtered_dff_minute_novel2": filtered_dff_minute_novel2,#dff data
    "filtered_resampled_dff_minute_novel": filtered_resampled_dff_minute_novel,#dff data
    "filtered_dff_minute_Rfamiliar": filtered_dff_minute_Rfamiliar,#velocity data
    "filtered_dff_minute_Rfamiliar2": filtered_dff_minute_Rfamiliar2,#dff data
    "filtered_resampled_dff_minute_Rfamiliar": filtered_resampled_dff_minute_Rfamiliar,#dff data
    "filtered_resampled_velocity_novel":filtered_resampled_velocity_novel,#velocity data
    "filtered_resampled_velocity_Rfamiliar":filtered_resampled_velocity_Rfamiliar,#velocity data
    "resampled_filtered_novel_all":resampled_filtered_novel_all,#dff data
    "resampled_filtered_novel_indices_all": resampled_filtered_novel_indices_all,#indices
    "resampled_filtered_Rfamiliar_all":resampled_filtered_Rfamiliar_all,#dff data
    "resampled_filtered_Rfamiliar_indices_all":resampled_filtered_Rfamiliar_indices_all,#indices
    "running_index_familiar_list":running_index_familiar_list, #index
    "running_index_novel_list": running_index_novel_list, #index
    "running_index_Rfamiliar_list":running_index_Rfamiliar_list, #index
    "speed_familiar_all":speed_familiar_all#velocity data

    
}

# Loop through the dictionary and save each dataset to its corresponding file
for filename, data in data_to_save.items():
    with open(filename, "wb") as fp:
        pickle.dump(data, fp)


In [ ]:
import pickle

# Dictionary to store the loaded data
data_loaded = {}

# List of file names
file_names = [
    "filtered_dff_minute_familiar",
    "filtered_dff_minute_ave_familiar",
    "filtered_dff_minute_novel",
    "filtered_dff_minute_novel2",
    "filtered_resampled_dff_minute_novel",
    "filtered_dff_minute_Rfamiliar",
    "filtered_dff_minute_Rfamiliar2",
    "filtered_resampled_dff_minute_Rfamiliar",
    "filtered_resampled_velocity_novel",
    "filtered_resampled_velocity_Rfamiliar",
    "resampled_filtered_novel_all",
    "resampled_filtered_novel_indices_all",
    "resampled_filtered_Rfamiliar_all",
    "resampled_filtered_Rfamiliar_indices_all",
    "running_index_familiar_list",
    "running_index_novel_list",
    "running_index_Rfamiliar_list",
    "speed_familiar_all"
]

# Loop through the file names and load each file
for filename in file_names:
    with open(filename, "rb") as fp:
        data_loaded[filename] = pickle.load(fp)

# access each loaded data with data_loaded["filename"]
filtered_dff_minute_familiar = data_loaded["filtered_dff_minute_familiar"]#dff data
filtered_dff_minute_ave_familiar = data_loaded["filtered_dff_minute_ave_familiar"]#dff data
filtered_dff_minute_novel = data_loaded["filtered_dff_minute_novel"]#velocity data
filtered_dff_minute_novel2 = data_loaded["filtered_dff_minute_novel2"]#dff data
filtered_resampled_dff_minute_novel = data_loaded["filtered_resampled_dff_minute_novel"]#dff data
filtered_dff_minute_Rfamiliar = data_loaded["filtered_dff_minute_Rfamiliar"]#velocity data
filtered_dff_minute_Rfamiliar2 = data_loaded["filtered_dff_minute_Rfamiliar2"]#dff data
filtered_resampled_dff_minute_Rfamiliar = data_loaded["filtered_resampled_dff_minute_Rfamiliar"]#dff data
filtered_resampled_velocity_novel = data_loaded["filtered_resampled_velocity_novel"]#velocity data
filtered_resampled_velocity_Rfamiliar = data_loaded["filtered_resampled_velocity_Rfamiliar"]#velocity data
resampled_filtered_novel_all = data_loaded["resampled_filtered_novel_all"]#dff data
resampled_filtered_novel_indices_all = data_loaded["resampled_filtered_novel_indices_all"]#indices
resampled_filtered_Rfamiliar_all = data_loaded["resampled_filtered_Rfamiliar_all"]#dff data
resampled_filtered_Rfamiliar_indices_all = data_loaded["resampled_filtered_Rfamiliar_indices_all"]#indices
running_index_familiar_list = data_loaded["running_index_familiar_list"] #index
running_index_novel_list = data_loaded["running_index_novel_list"] #index
running_index_Rfamiliar_list = data_loaded["running_index_Rfamiliar_list"] #index
speed_familiar_all = data_loaded["speed_familiar_all"]#velocity data


### Inspection of resample by plotting 

In [ ]:
#resample data

import seaborn as sns
import matplotlib.pyplot as plt

# Number of rows (i) and columns (n) you want
rows = len(speed_familiar_all)  # Assuming i represents the number of entries in speed_familiar_all
columns = 10  # n is set to 10 based on your description

# Create a figure with a grid of subplots
fig, axes = plt.subplots(rows, columns, figsize=(columns * 5, rows * 3))  # Adjust figsize as needed

# Plot the KDEs in each subplot
for i in range(rows):
    for n in range(columns):
        ax = axes[i, n]  # Get the current subplot
        
        # Plot the KDEs
        sns.kdeplot(speed_familiar_all[i], color='grey', linewidth=2, linestyle='-', label='Familiar', ax=ax)
        sns.kdeplot(filtered_resampled_velocity_novel[i][n], color='red', linewidth=2, linestyle='-', label='Novel', ax=ax)
        sns.kdeplot(filtered_resampled_velocity_Rfamiliar[i][n], color='black', linewidth=2, linestyle='-', label='Rfamiliar', ax=ax)
        
        # Customize the subplot appearance
        # Remove x-ticks and y-ticks
        ax.set_xticks([])  # Remove x-ticks
        ax.set_yticks([])  # Remove y-ticks

        # Remove axis labels
        ax.set_xlabel('')  # Remove x-axis label
        ax.set_ylabel('')  # Remove y-axis label
        
        # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Change border color and thickness
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(2)

        # Optionally remove legend for cleaner appearance
        # ax.legend(loc='upper right', fontsize=10, frameon=False)

# Adjust layout so subplots fit nicely
plt.tight_layout()

# Save the plot with high resolution (DPI)
#plt.savefig("filtered_resampled_velocity.png", dpi=600)  # Increase DPI for higher resolution

# Show the plot
plt.show()


In [ ]:
#Filtered data before resample

import seaborn as sns
import matplotlib.pyplot as plt

# Number of rows (i) and columns (n) you want
rows = len(speed_familiar_all)  # Assuming i represents the number of entries in speed_familiar_all
columns = 10  # n is set to 10 based on your description

# Create a figure with a grid of subplots
fig, axes = plt.subplots(rows, columns, figsize=(columns * 5, rows * 3))  # Adjust figsize as needed

# Plot the KDEs in each subplot
for i in range(rows):
    for n in range(columns):
        ax = axes[i, n]  # Get the current subplot
        
        # Plot the KDEs
        sns.kdeplot(speed_familiar_all[i], color='grey', linewidth=2, linestyle='-', label='Familiar', ax=ax)
        sns.kdeplot(filtered_dff_minute_novel[i][n], color='red', linewidth=2, linestyle='-', label='Novel', ax=ax)
        sns.kdeplot(filtered_dff_minute_Rfamiliar[i][n], color='black', linewidth=2, linestyle='-', label='Rfamiliar', ax=ax)
        
        # Customize the subplot appearance
        # Remove x-ticks and y-ticks
        ax.set_xticks([])  # Remove x-ticks
        ax.set_yticks([])  # Remove y-ticks

        # Remove axis labels
        ax.set_xlabel('')  # Remove x-axis label
        ax.set_ylabel('')  # Remove y-axis label
        
        # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Change border color and thickness
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(2)

        # Optionally remove legend for cleaner appearance
        # ax.legend(loc='upper right', fontsize=10, frameon=False)

# Adjust layout so subplots fit nicely
plt.tight_layout()

# Save the plot with high resolution (DPI)
#plt.savefig("filtered_velocity_sit_thresh1.png", dpi=600)  # Increase DPI for higher resolution

# Show the plot
plt.show()


In [ ]:
velocity_familiar_all=[]
velocity_minute_novel=[]
velocity_minute_Rfamiliar=[]
velocity_minute_familiar=[]
velocity_minute_familiar
dff_minute_novel=[]
dff_minute_Rfamiliar=[]
for i in range(0,len(df_downsample)):
    velocity_f = savgol_filter(df_downsample[i].Velocity,50,3)
    velocity_f=pd.DataFrame(velocity_f)
    velocity_f_positive=velocity_f[0]-velocity_f[0].min()+0.0000001
    switch1=switch[i].switch1.iloc[0]
    switch2=switch[i].switch2.iloc[0]
    # Filter running speed
    speed_familiar= np.array(velocity_f_positive[:switch1])

    speed_novel= np.array(velocity_f_positive[switch1:switch2])#could use"int(switch1+(switch2-switch1)/2)" insteatd of "switch2"

    speed_Rfamiliar= np.array(velocity_f_positive[switch2:])

    velocity_familiar_all=velocity_familiar_all+[speed_familiar]
    
    df0=df_downsample[i][:switch1]
    df0=df0.reset_index().drop(columns="index")
    df0.Time=df0.Time-df0.Time[0]

    velocity_average_minute_familiar_each=[]
    for n in range(1,10+1):
        if n == 1:
            # Compute the absolute difference between the 'Time' column and 1
            abs_diff = np.abs(df0.Time - n*60)
            # Find the index of the minimum value in the absolute difference series
            X = np.argmin(abs_diff)
            velocity_minute=dff_f_mean[i][:switch1][:X]
            velocity_average_minute_familiar_each.append(velocity_minute) 
        else:
            abs_diff1 = np.abs(df0.Time - (n-1)*60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(df0.Time - n*60)
            X2 = np.argmin(abs_diff2)
            velocity_minute=dff_f_mean[i][:switch1][X1:X2]
            velocity_average_minute_familiar_each.append(velocity_minute)
    velocity_minute_familiar += [velocity_average_minute_familiar_each]
    
    df1=df_downsample[i][switch1:switch2]
    df1=df1.reset_index().drop(columns="index")
    df1.Time=df1.Time-df1.Time[0]
    #velocity_f = savgol_filter(df_downsample[i].Velocity,50,3)
    #velocity_f=pd.DataFrame(velocity_f)
    #velocity_f_positive=velocity_f[0]-velocity_f[0].min()+0.0000001
    velocity_f_positive_novel=velocity_f_positive[switch1:switch2]
    velocity_f_positive_novel=velocity_f_positive_novel.reset_index().drop(columns="index")
    velocity_f_positive_novel=np.array(velocity_f_positive_novel).ravel()
    
    velocity_average_minute_novel_each=[]
    dff_average_minute_novel_each=[]
    for n in range(1,20+1):
        if n == 1:
            # Compute the absolute difference between the 'Time' column and 1
            abs_diff = np.abs(df1.Time - n*60)
            # Find the index of the minimum value in the absolute difference series
            X = np.argmin(abs_diff)
            velocity_minute=velocity_f_positive_novel[:X]
            velocity_average_minute_novel_each.append(velocity_minute) 
            dff_minute=dff_f_mean[i][switch1:switch2][:X]
            dff_average_minute_novel_each.append(dff_minute) 
        else:
            abs_diff1 = np.abs(df1.Time - (n-1)*60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(df1.Time - n*60)
            X2 = np.argmin(abs_diff2)
            velocity_minute=velocity_f_positive_novel[X1:X2]
            velocity_average_minute_novel_each.append(velocity_minute) 
            dff_minute=dff_f_mean[i][switch1:switch2][X1:X2]
            dff_average_minute_novel_each.append(dff_minute) 
    velocity_minute_novel += [velocity_average_minute_novel_each]
    dff_minute_novel += [dff_average_minute_novel_each]
    
    df2=df_downsample[i][switch2:]
    df2=df2.reset_index().drop(columns="index")
    df2.Time=df2.Time-df2.Time[0]
    velocity_f_positive_Rfamiliar=velocity_f_positive[switch2:]
    velocity_f_positive_Rfamiliar=velocity_f_positive_Rfamiliar.reset_index().drop(columns="index")
    velocity_f_positive_Rfamiliar=np.array(velocity_f_positive_Rfamiliar).ravel()
   
    velocity_average_minute_Rfamiliar_each=[]
    dff_average_minute_Rfamiliar_each=[]
    for n in range(1,10+1):
        if n == 1:
            # Compute the absolute difference between the 'Time' column and 1
            abs_diff = np.abs(df2.Time - n*60)
            # Find the index of the minimum value in the absolute difference series
            X = np.argmin(abs_diff)
            velocity_minute=velocity_f_positive_Rfamiliar[:X]
            velocity_average_minute_Rfamiliar_each.append(velocity_minute) 
            dff_minute=dff_f_mean[i][switch2:][:X]
            dff_average_minute_Rfamiliar_each.append(dff_minute) 
        else:
            abs_diff1 = np.abs(df2.Time - (n-1)*60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(df2.Time - n*60)
            X2 = np.argmin(abs_diff2)
            velocity_minute=velocity_f_positive_Rfamiliar[X1:X2]
            velocity_average_minute_Rfamiliar_each.append(velocity_minute)
            dff_minute=dff_f_mean[i][switch2:][X1:X2]
            dff_average_minute_Rfamiliar_each.append(dff_minute) 
    velocity_minute_Rfamiliar += [velocity_average_minute_Rfamiliar_each]
    dff_minute_Rfamiliar += [dff_average_minute_Rfamiliar_each]

In [ ]:
#Data before filter and resample

import seaborn as sns
import matplotlib.pyplot as plt

# Number of rows (i) and columns (n) you want
rows = len(velocity_familiar_all)  # Assuming i represents the number of entries in speed_familiar_all
columns = 10  # n is set to 10 based on your description

# Create a figure with a grid of subplots
fig, axes = plt.subplots(rows, columns, figsize=(columns * 5, rows * 3))  # Adjust figsize as needed

# Plot the KDEs in each subplot
for i in range(rows):
    for n in range(columns):
        ax = axes[i, n]  # Get the current subplot
        
        # Plot the KDEs
        sns.kdeplot(velocity_familiar_all[i], color='grey', linewidth=2, linestyle='-', label='Familiar', ax=ax)
        sns.kdeplot(velocity_minute_novel[i][n], color='red', linewidth=2, linestyle='-', label='Novel', ax=ax)
        sns.kdeplot(velocity_minute_Rfamiliar[i][n], color='black', linewidth=2, linestyle='-', label='Rfamiliar', ax=ax)
        
        # Customize the subplot appearance
        # Remove x-ticks and y-ticks
        ax.set_xticks([])  # Remove x-ticks
        ax.set_yticks([])  # Remove y-ticks

        # Remove axis labels
        ax.set_xlabel('')  # Remove x-axis label
        ax.set_ylabel('')  # Remove y-axis label
        
        # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Change border color and thickness
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(2)

        # Optionally remove legend for cleaner appearance
        # ax.legend(loc='upper right', fontsize=10, frameon=False)

# Adjust layout so subplots fit nicely
plt.tight_layout()

# Save the plot with high resolution (DPI)
#plt.savefig("unfiltered_velocity.png", dpi=600)  # Increase DPI for higher resolution

# Show the plot
plt.show()


### Wilocoxon test

In [ ]:
from scipy.stats import wilcoxon

# Convert lists to numpy arrays if not already arrays and handle None values by converting them to np.nan
speed_average_minute_novel = np.array(filtered_resampled_dff_minute_novel, dtype=np.float64)
speed_average_minute_Rfamiliar = np.array(filtered_resampled_dff_minute_Rfamiliar, dtype=np.float64)
speed_average_minute_familiar = np.mean(filtered_dff_minute_ave_familiar, axis=1)

# Ensure all arrays are of shape (N, 10) where N is the number of samples
speed_average_minute_novel = speed_average_minute_novel[:, :10]
speed_average_minute_Rfamiliar = speed_average_minute_Rfamiliar[:, :10]

# Initialize lists to store p-values
p_values_novel = []
p_values_Rfamiliar = []
p_values_Rnovel = []
len_familiar=[]
len_novel=[]
len_Rfamiliar=[]
excluded_familiar=[]
excluded_novel=[]
excluded_Rfamiliar=[]

for i in range(10):
    # Extract data for the current time point, excluding rows with NaN in any group
    novel_time_point = speed_average_minute_novel[:, i]
    Rfamiliar_time_point = speed_average_minute_Rfamiliar[:, i]
    
    # Create a mask to identify rows without NaN values across all groups for the current time point
    valid_mask = ~np.isnan(speed_average_minute_familiar) & ~np.isnan(novel_time_point) & ~np.isnan(Rfamiliar_time_point)
    
    # Apply the mask to keep only rows with valid data in all groups
    filtered_familiar = speed_average_minute_familiar[valid_mask]
    filtered_novel = novel_time_point[valid_mask]
    filtered_Rfamiliar = Rfamiliar_time_point[valid_mask]
    
    # Compare familiar vs novel
    stat, p = wilcoxon(filtered_familiar, filtered_novel)
    p_values_novel.append(p)
    
    # Compare familiar vs returned familiar
    stat, p = wilcoxon(filtered_familiar, filtered_Rfamiliar)
    p_values_Rfamiliar.append(p)
    
    stat, p = wilcoxon(filtered_novel, filtered_Rfamiliar)
    p_values_Rnovel.append(p)
    
    len_familiar.append(len(filtered_familiar))
    len_novel.append(len(filtered_novel))
    len_Rfamiliar.append(len(filtered_Rfamiliar))
    excluded_familiar.append(filtered_familiar.tolist())
    excluded_novel.append(filtered_novel.tolist())
    excluded_Rfamiliar.append(filtered_Rfamiliar.tolist())
    

# Print the p-values
print("P-values for familiar vs novel at each time point:", p_values_novel)
print("P-values for familiar vs returned familiar at each time point:", p_values_Rfamiliar)
print("P-values for novel vs returned familiar at each time point:", p_values_Rnovel)
print('F_length', len_familiar)
print('N_length', len_novel)
print('R_length', len_Rfamiliar)

In [ ]:
from scipy.stats import wilcoxon

# Convert lists to numpy arrays if not already arrays and handle None values by converting them to np.nan
speed_average_minute_novel = np.array(filtered_resampled_dff_minute_novel, dtype=np.float64)
speed_average_minute_Rfamiliar = np.array(filtered_resampled_dff_minute_Rfamiliar, dtype=np.float64)
speed_average_minute_familiar = np.mean(filtered_dff_minute_ave_familiar, axis=1)

# Ensure all arrays are of shape (N, 10) where N is the number of samples
speed_average_minute_novel = speed_average_minute_novel[:, :10]
speed_average_minute_Rfamiliar = speed_average_minute_Rfamiliar[:, :10]

# Initialize lists to store p-values
p_values_novel = []
p_values_Rfamiliar = []
p_values_Rnovel = []
len_familiar=[]
len_novel=[]
len_Rfamiliar=[]

i = 0
    # Extract data for the current time point, excluding rows with NaN in any group
novel_time_point = speed_average_minute_novel[1:, i] #remove C7 data
Rfamiliar_time_point = speed_average_minute_Rfamiliar[1:, i] #remove C7 data


    # Create a mask to identify rows without NaN values across all groups for the current time point
valid_mask = ~np.isnan(speed_average_minute_familiar[1:]) & ~np.isnan(novel_time_point) & ~np.isnan(Rfamiliar_time_point)
    
    # Apply the mask to keep only rows with valid data in all groups
filtered_familiar = speed_average_minute_familiar[1:][valid_mask]
filtered_novel = novel_time_point[valid_mask]
filtered_Rfamiliar = Rfamiliar_time_point[valid_mask]
    
    # Compare familiar vs novel
stat, p = wilcoxon(filtered_familiar, filtered_novel)
p_values_novel.append(p)
    
    # Compare familiar vs returned familiar
stat, p = wilcoxon(filtered_familiar, filtered_Rfamiliar)
p_values_Rfamiliar.append(p)
    
stat, p = wilcoxon(filtered_novel, filtered_Rfamiliar)
p_values_Rnovel.append(p)
    
len_familiar.append(len(filtered_familiar))
len_novel.append(len(filtered_novel))
len_Rfamiliar.append(len(filtered_Rfamiliar))
    

# Print the p-values
print("P-values for familiar vs novel at each time point:", p_values_novel)
print("P-values for familiar vs returned familiar at each time point:", p_values_Rfamiliar)
print("P-values for novel vs returned familiar at each time point:", p_values_Rnovel)
print('F_length', len_familiar)
print('N_length', len_novel)
print('R_length', len_Rfamiliar)

In [ ]:
from scipy.stats import wilcoxon

# Convert lists to numpy arrays if not already arrays and handle None values by converting them to np.nan
speed_average_minute_novel = np.array(filtered_resampled_dff_minute_novel, dtype=np.float64)

excluded_novel2=[]

for i in range(10,20):
    # Extract data for the current time point, excluding rows with NaN in any group
    novel_time_point = speed_average_minute_novel[:, i]
    
    # Create a mask to identify rows without NaN values across all groups for the current time point
    valid_mask = ~np.isnan(novel_time_point)
 
    filtered_novel = novel_time_point[valid_mask]

    excluded_novel2.append(filtered_novel.tolist())
    
# Print
print(len(excluded_novel2))

In [ ]:
len(excluded_novel+excluded_novel2)

## plotting data 

In [ ]:
# Legend labels
legend_labels = ["novel", "returned familiar"]

# Prepare data
excluded_novel20=excluded_novel+excluded_novel2 #20min data
excluded_familiar20=excluded_familiar+excluded_familiar #20 min data

increased_familiar = [[familiar - familiar for familiar, familiar in zip(row_familiar, row_familiar)]
for row_familiar, row_familiar in zip(excluded_familiar20, excluded_familiar20)]
increased_novel = [[novel - familiar for novel, familiar in zip(row_novel, row_familiar)]
for row_novel, row_familiar in zip(excluded_novel20, excluded_familiar20)]
increased_Rfamiliar = [[Rfamiliar - familiar for Rfamiliar, familiar in zip(row_Rfamiliar, row_familiar)]
for row_Rfamiliar, row_familiar in zip(excluded_Rfamiliar, excluded_familiar)]

mean_familiar = []
mean_novel = []
mean_Rfamiliar = []
ste_familiar = []
ste_novel = []
ste_Rfamiliar = []

# Assuming each increased_* is a list of lists, we need to iterate through each sublist
for i in range(10):
    mean_familiar.append(np.mean(increased_familiar[i]))
    mean_Rfamiliar.append(np.mean(increased_Rfamiliar[i]))
    ste_familiar.append(np.std(increased_familiar[i]) / np.sqrt(len(increased_familiar[i])))
    ste_Rfamiliar.append(np.std(increased_Rfamiliar[i]) / np.sqrt(len(increased_Rfamiliar[i])))
for i in range(20):    
    mean_novel.append(np.mean(increased_novel[i]))
    ste_novel.append(np.std(increased_novel[i]) / np.sqrt(len(increased_novel[i])))

# Combine the filtered data lists for easier iteration
mean_data_lists = [mean_novel, mean_Rfamiliar]
ste_data_lists = [ste_novel, ste_Rfamiliar]

# Define colors for each criterion
colors = ["red", "black"]

# Define lighter colors for filling
lighter_colors = ["#FFB6C1", "#A9A9A9"]

# Prepare figure
plt.figure(figsize=(9, 6))

# Iterate over each data list to plot
for i, (avg_data, std_error) in enumerate(zip(mean_data_lists, ste_data_lists)):
    # Convert mean and std error lists to NumPy arrays
    avg_data = np.array(avg_data)
    std_error = np.array(std_error)

    # Define x-values specific to the length of avg_data
    x_values = np.arange(1, len(avg_data) + 1)

    # Plot average data
    plt.plot(x_values, avg_data, label=legend_labels[i], color=colors[i], linewidth=2.5)

    # Plot standard error as filled area
    plt.fill_between(x_values, avg_data - std_error, avg_data + std_error, color=lighter_colors[i], alpha=0.5)

# Plot familiar_mean as a dashed line (without error bars)
familiar_mean_avg = np.mean(mean_familiar)
plt.plot(range(1, len(mean_data_lists[0]) + 1), [familiar_mean_avg] * len(mean_data_lists[0]), 
         label="familiar_mean", color="grey", linestyle="--", linewidth=4)

# Adding titles and labels
plt.title('resampled_filtered_dF/F', fontsize=22)
plt.xlabel('Time (min)', fontsize=22)
plt.ylabel('ΔF/F increased from familiar (%)', fontsize=22)

# Set x-ticks
plt.xticks(ticks=[0, 5, 10, 15, 20], fontsize=18)
plt.yticks(fontsize=18)

# Change the thickness of x-ticks and y-ticks
plt.tick_params(axis='both', width=2)

# Adding a legend outside the plot
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=18)

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')
    spine.set_linewidth(2)

# Layout adjustment to accommodate legend
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
# Legend labels
legend_labels = ["novel", "returned familiar"]

# Prepare data
excluded_novel20=excluded_novel+excluded_novel2 #20min data
excluded_familiar20=excluded_familiar+excluded_familiar #20 min data

mean_familiar = []
mean_novel = []
mean_Rfamiliar = []
ste_familiar = []
ste_novel = []
ste_Rfamiliar = []

# Assuming each increased_* is a list of lists, we need to iterate through each sublist
for i in range(10):
    mean_familiar.append(np.mean(excluded_familiar20[i]))
    mean_Rfamiliar.append(np.mean(excluded_Rfamiliar[i]))
    ste_familiar.append(np.std(excluded_familiar20[i]) / np.sqrt(len(excluded_familiar20[i])))
    ste_Rfamiliar.append(np.std(excluded_Rfamiliar[i]) / np.sqrt(len(excluded_Rfamiliar[i])))
for i in range(20):    
    mean_novel.append(np.mean(excluded_novel20[i]))
    ste_novel.append(np.std(excluded_novel20[i]) / np.sqrt(len(excluded_novel20[i])))

# Combine the filtered data lists for easier iteration
mean_data_lists = [mean_novel, mean_Rfamiliar]
ste_data_lists = [ste_novel, ste_Rfamiliar]

# Define colors for each criterion
colors = ["red", "black"]

# Define lighter colors for filling
lighter_colors = ["#FFB6C1", "#A9A9A9"]

# Prepare figure
plt.figure(figsize=(9, 6))

# Iterate over each data list to plot
for i, (avg_data, std_error) in enumerate(zip(mean_data_lists, ste_data_lists)):
    # Convert mean and std error lists to NumPy arrays
    avg_data = np.array(avg_data)
    std_error = np.array(std_error)

    # Define x-values specific to the length of avg_data
    x_values = np.arange(1, len(avg_data) + 1)

    # Plot average data
    plt.plot(x_values, avg_data, label=legend_labels[i], color=colors[i], linewidth=2.5)

    # Plot standard error as filled area
    plt.fill_between(x_values, avg_data - std_error, avg_data + std_error, color=lighter_colors[i], alpha=0.5)

# Plot familiar_mean as a dashed line (without error bars)
familiar_mean_avg = np.mean(mean_familiar)
plt.plot(range(1, len(mean_data_lists[0]) + 1), [familiar_mean_avg] * len(mean_data_lists[0]), 
         label="familiar_mean", color="grey", linestyle="--", linewidth=4)

# Adding titles and labels
plt.title('resampled_filtered_dF/F', fontsize=22)
plt.xlabel('Time (min)', fontsize=22)
plt.ylabel('ΔF/F (%)', fontsize=22)

# Set x-ticks
plt.xticks(ticks=[0, 5, 10, 15, 20], fontsize=18)
plt.yticks(fontsize=18)

# Change the thickness of x-ticks and y-ticks
plt.tick_params(axis='both', width=2)

# Adding a legend outside the plot
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=18)

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')
    spine.set_linewidth(2)

# Layout adjustment to accommodate legend
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
# dff change at 1st min

# Convert lists to numpy arrays
speed_average_minute_novel = np.array(filtered_resampled_dff_minute_novel, dtype=np.float64)
speed_average_minute_Rfamiliar = np.array(filtered_resampled_dff_minute_Rfamiliar, dtype=np.float64)
speed_average_minute_familiar = np.mean(filtered_dff_minute_ave_familiar, axis=1)

# Ensure all arrays are of shape (N, 10) where N is the number of samples
speed_average_minute_novel = speed_average_minute_novel[:, :10]
speed_average_minute_Rfamiliar = speed_average_minute_Rfamiliar[:, :10]

i=0
    # Extract data for the current time point, excluding rows with NaN in any group
novel_time_point = speed_average_minute_novel[:, i] 
Rfamiliar_time_point = speed_average_minute_Rfamiliar[:, i] 
    
    # Create a mask to identify rows without NaN values across all groups for the current time point
valid_mask = ~np.isnan(speed_average_minute_familiar) & ~np.isnan(novel_time_point) & ~np.isnan(Rfamiliar_time_point)
    
    # Apply the mask to keep only rows with valid data in all groups
filtered_familiar = speed_average_minute_familiar[valid_mask]
filtered_novel = novel_time_point[valid_mask]
filtered_Rfamiliar = Rfamiliar_time_point[valid_mask]

novel_familiar=filtered_novel-filtered_familiar
Rfamiliar_familiar=filtered_Rfamiliar-filtered_familiar
familiar_familiar=filtered_familiar-filtered_familiar
# Calculate mean values
average_familiar = np.mean(familiar_familiar)
average_novel = np.mean(novel_familiar)
average_Rfamiliar = np.mean(Rfamiliar_familiar)

# Prepare data for bar plot
group_labels = ['Familiar','Novel', 'Rfamiliar']
mean_values = [average_familiar,average_novel, average_Rfamiliar]

# Adjust x-positions to give more space between the groups
x_positions = [0.98, 1.01, 1.04]  # Reduce the distance between the two bars

# Plot the bar plot, scatter plots, and dashed line
plt.figure(figsize=(6.6, 6.6))
for i in range(len(novel_familiar)):
    plt.plot([x_positions[0], x_positions[1]], 
             [familiar_familiar[i], novel_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)
    # Add the index label next to each point
    #plt.text(x_positions[1] - 0.0001, xcorr_minute_novel_first_min[i], str(i+1), fontsize=12, color='black')
    plt.plot([x_positions[1], x_positions[2]], 
             [novel_familiar[i], Rfamiliar_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)

# Scatter plot for familiar group
plt.scatter([x_positions[0]] * len(familiar_familiar), 
            familiar_familiar, s=100, color='grey', alpha=0.7, label='Familiar')    
    
# Scatter plot for Novel group
plt.scatter([x_positions[1]] * len(novel_familiar), 
            novel_familiar, s=100, color='red', alpha=0.7, label='Novel')

# Scatter plot for R-Familiar group
plt.scatter([x_positions[2]] * len(Rfamiliar_familiar), 
            Rfamiliar_familiar, s=100, color='black', alpha=0.7, label='Rfamiliar')

# Add a dashed line at the mean value for Familiar group across the entire plot
#plt.axhline(y=average_xcorr_familiar, color='grey', linestyle='--', linewidth=2, label='Mean Familiar')

# Add labels and title
plt.ylabel('ΔF/F increased from familiar (%)', size=21)
plt.title('1st minute', fontsize=21, pad=10)

# Set y-limits to make the plot cleaner
plt.ylim(-1.7, 9)
plt.xlim(0.97,1.05)

# Set custom x-ticks for the two groups (Novel, Rfamiliar)
plt.xticks(x_positions, ['Familiar(Avg)','Novel', 'Rfamiliar'], fontsize=20)
plt.yticks([0, 2, 4, 6, 8], fontsize=20)

# Move the legend outside the plot and remove border
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, frameon=False)

# Remove top and right spines (border)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')  # Change border color to black
    spine.set_linewidth(2)     # Increase border thickness

# Show plot
plt.grid(False)
plt.tight_layout()
plt.show()



In [ ]:
# dff change at 1st min with each label

# Convert lists to numpy arrays 
speed_average_minute_novel = np.array(filtered_resampled_dff_minute_novel, dtype=np.float64)
speed_average_minute_Rfamiliar = np.array(filtered_resampled_dff_minute_Rfamiliar, dtype=np.float64)
speed_average_minute_familiar = np.mean(filtered_dff_minute_ave_familiar, axis=1)

# Ensure all arrays are of shape (N, 10) where N is the number of samples
speed_average_minute_novel = speed_average_minute_novel[:, :10]
speed_average_minute_Rfamiliar = speed_average_minute_Rfamiliar[:, :10]

i=0
    # Extract data for the current time point, excluding rows with NaN in any group
novel_time_point = speed_average_minute_novel[1:, i]
Rfamiliar_time_point = speed_average_minute_Rfamiliar[1:, i]
    
    # Create a mask to identify rows without NaN values across all groups for the current time point
valid_mask = ~np.isnan(speed_average_minute_familiar[1:]) & ~np.isnan(novel_time_point) & ~np.isnan(Rfamiliar_time_point)
    
    # Apply the mask to keep only rows with valid data in all groups
filtered_familiar = speed_average_minute_familiar[1:][valid_mask]
filtered_novel = novel_time_point[valid_mask]
filtered_Rfamiliar = Rfamiliar_time_point[valid_mask]

novel_familiar=filtered_novel-filtered_familiar
Rfamiliar_familiar=filtered_Rfamiliar-filtered_familiar
familiar_familiar=filtered_familiar-filtered_familiar
# Calculate mean values
average_familiar = np.mean(familiar_familiar)
average_novel = np.mean(novel_familiar)
average_Rfamiliar = np.mean(Rfamiliar_familiar)

# Prepare data for bar plot
group_labels = ['Familiar','Novel', 'Rfamiliar']
mean_values = [average_familiar,average_novel, average_Rfamiliar]

# Adjust x-positions to give more space between the groups
x_positions = [0.98, 1.01, 1.04]  # Reduce the distance between the two bars

# Plot the bar plot, scatter plots, and dashed line
plt.figure(figsize=(8, 8))
for i in range(len(novel_familiar)):
    plt.plot([x_positions[0], x_positions[1]], 
             [familiar_familiar[i], novel_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)
    # Add the index label next to each point
    plt.plot([x_positions[1], x_positions[2]], 
             [novel_familiar[i], Rfamiliar_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)
    
        # Add the index label next to each point for each group
    plt.text(x_positions[2] + 0.002, Rfamiliar_familiar[i], str(i + 1), fontsize=9, color='black')

# Scatter plot for familiar group
plt.scatter([x_positions[0]] * len(familiar_familiar), 
            familiar_familiar, s=100, color='grey', alpha=0.7, label='Familiar')
    
# Scatter plot for Novel group
plt.scatter([x_positions[1]] * len(novel_familiar), 
            novel_familiar, s=100, color='red', alpha=0.7, label='Novel')

# Scatter plot for R-Familiar group
plt.scatter([x_positions[2]] * len(Rfamiliar_familiar), 
            Rfamiliar_familiar, s=100, color='black', alpha=0.7, label='Rfamiliar')

# Add a dashed line at the mean value for Familiar group across the entire plot
#plt.axhline(y=average_xcorr_familiar, color='grey', linestyle='--', linewidth=2, label='Mean Familiar')

# Add labels and title
plt.ylabel('ΔF/F increased from familiar (%)', size=21)
plt.title('1st minute', fontsize=21, pad=40)

# Set y-limits to make the plot cleaner
plt.ylim(-1.7, 9)
plt.xlim(0.97,1.05)

# Set custom x-ticks for the two groups (Novel, Rfamiliar)
plt.xticks(x_positions, ['Familiar (Avg)','Novel', 'Rfamiliar'], fontsize=18)
plt.yticks([0, 2, 4, 6, 8], fontsize=18)

# Move the legend outside the plot and remove border
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, frameon=False)

# Remove top and right spines (border)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')  # Change border color to black
    spine.set_linewidth(2)     # Increase border thickness

# Show plot
plt.grid(False)
plt.tight_layout()
plt.show()



In [ ]:
# dff change at 5th min

# Convert lists to numpy arrays 
speed_average_minute_novel = np.array(filtered_resampled_dff_minute_novel, dtype=np.float64)
speed_average_minute_Rfamiliar = np.array(filtered_resampled_dff_minute_Rfamiliar, dtype=np.float64)
speed_average_minute_familiar = np.mean(filtered_dff_minute_ave_familiar, axis=1)

# Ensure all arrays are of shape (N, 10) where N is the number of samples
speed_average_minute_novel = speed_average_minute_novel[:, :10]
speed_average_minute_Rfamiliar = speed_average_minute_Rfamiliar[:, :10]

i=4
    # Extract data for the current time point, excluding rows with NaN in any group
novel_time_point = speed_average_minute_novel[:, i] 
Rfamiliar_time_point = speed_average_minute_Rfamiliar[:, i] 
    
    # Create a mask to identify rows without NaN values across all groups for the current time point
valid_mask = ~np.isnan(speed_average_minute_familiar) & ~np.isnan(novel_time_point) & ~np.isnan(Rfamiliar_time_point)
    
    # Apply the mask to keep only rows with valid data in all groups
filtered_familiar = speed_average_minute_familiar[valid_mask]
filtered_novel = novel_time_point[valid_mask]
filtered_Rfamiliar = Rfamiliar_time_point[valid_mask]

novel_familiar=filtered_novel-filtered_familiar
Rfamiliar_familiar=filtered_Rfamiliar-filtered_familiar
familiar_familiar=filtered_familiar-filtered_familiar
# Calculate mean values
average_familiar = np.mean(familiar_familiar)
average_novel = np.mean(novel_familiar)
average_Rfamiliar = np.mean(Rfamiliar_familiar)

# Prepare data for bar plot
group_labels = ['Familiar','Novel', 'Rfamiliar']
mean_values = [average_familiar,average_novel, average_Rfamiliar]

# Adjust x-positions to give more space between the groups
x_positions = [0.98, 1.01, 1.04]  # Reduce the distance between the two bars

# Plot the bar plot, scatter plots, and dashed line
plt.figure(figsize=(6.6, 6.6))
for i in range(len(novel_familiar)):
    plt.plot([x_positions[0], x_positions[1]], 
             [familiar_familiar[i], novel_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)
    # Add the index label next to each point
    #plt.text(x_positions[1] - 0.0001, xcorr_minute_novel_first_min[i], str(i+1), fontsize=12, color='black')
    plt.plot([x_positions[1], x_positions[2]], 
             [novel_familiar[i], Rfamiliar_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)

# Scatter plot for familiar group
plt.scatter([x_positions[0]] * len(familiar_familiar), 
            familiar_familiar, s=100, color='grey', alpha=0.7, label='Familiar')    
    
# Scatter plot for Novel group
plt.scatter([x_positions[1]] * len(novel_familiar), 
            novel_familiar, s=100, color='red', alpha=0.7, label='Novel')

# Scatter plot for R-Familiar group
plt.scatter([x_positions[2]] * len(Rfamiliar_familiar), 
            Rfamiliar_familiar, s=100, color='black', alpha=0.7, label='Rfamiliar')

# Add a dashed line at the mean value for Familiar group across the entire plot
#plt.axhline(y=average_xcorr_familiar, color='grey', linestyle='--', linewidth=2, label='Mean Familiar')

# Add labels and title
plt.ylabel('ΔF/F increased from familiar (%)', size=21)
plt.title('5th  minute', fontsize=21, pad=10)

# Set y-limits to make the plot cleaner
plt.ylim(-3.2, 9)
plt.xlim(0.97,1.05)

# Set custom x-ticks for the two groups (Novel, Rfamiliar)
plt.xticks(x_positions, ['Familiar(Avg)','Novel', 'Rfamiliar'], fontsize=20)
plt.yticks([-2,0, 2, 4, 6, 8], fontsize=20)

# Move the legend outside the plot and remove border
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, frameon=False)

# Remove top and right spines (border)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')  # Change border color to black
    spine.set_linewidth(2)     # Increase border thickness

# Show plot
plt.grid(False)
plt.tight_layout()
plt.show()



In [ ]:
# dff change at 10th min

# Convert lists to numpy arrays 
speed_average_minute_novel = np.array(filtered_resampled_dff_minute_novel, dtype=np.float64)
speed_average_minute_Rfamiliar = np.array(filtered_resampled_dff_minute_Rfamiliar, dtype=np.float64)
speed_average_minute_familiar = np.mean(filtered_dff_minute_ave_familiar, axis=1)

# Ensure all arrays are of shape (N, 10) where N is the number of samples
speed_average_minute_novel = speed_average_minute_novel[:, :10]
speed_average_minute_Rfamiliar = speed_average_minute_Rfamiliar[:, :10]

i=9
    # Extract data for the current time point, excluding rows with NaN in any group
novel_time_point = speed_average_minute_novel[:, i] 
Rfamiliar_time_point = speed_average_minute_Rfamiliar[:, i] 
    
    # Create a mask to identify rows without NaN values across all groups for the current time point
valid_mask = ~np.isnan(speed_average_minute_familiar) & ~np.isnan(novel_time_point) & ~np.isnan(Rfamiliar_time_point)
    
    # Apply the mask to keep only rows with valid data in all groups
filtered_familiar = speed_average_minute_familiar[valid_mask]
filtered_novel = novel_time_point[valid_mask]
filtered_Rfamiliar = Rfamiliar_time_point[valid_mask]

novel_familiar=filtered_novel-filtered_familiar
Rfamiliar_familiar=filtered_Rfamiliar-filtered_familiar
familiar_familiar=filtered_familiar-filtered_familiar
# Calculate mean values
average_familiar = np.mean(familiar_familiar)
average_novel = np.mean(novel_familiar)
average_Rfamiliar = np.mean(Rfamiliar_familiar)

# Prepare data for bar plot
group_labels = ['Familiar','Novel', 'Rfamiliar']
mean_values = [average_familiar,average_novel, average_Rfamiliar]

# Adjust x-positions to give more space between the groups
x_positions = [0.98, 1.01, 1.04]  # Reduce the distance between the two bars

# Plot the bar plot, scatter plots, and dashed line
plt.figure(figsize=(6.6, 6.6))
for i in range(len(novel_familiar)):
    plt.plot([x_positions[0], x_positions[1]], 
             [familiar_familiar[i], novel_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)
    # Add the index label next to each point
    #plt.text(x_positions[1] - 0.0001, xcorr_minute_novel_first_min[i], str(i+1), fontsize=12, color='black')
    plt.plot([x_positions[1], x_positions[2]], 
             [novel_familiar[i], Rfamiliar_familiar[i]], 
             color='grey', alpha=0.5, linewidth=2)

# Scatter plot for familiar group
plt.scatter([x_positions[0]] * len(familiar_familiar), 
            familiar_familiar, s=100, color='grey', alpha=0.7, label='Familiar')    
    
# Scatter plot for Novel group
plt.scatter([x_positions[1]] * len(novel_familiar), 
            novel_familiar, s=100, color='red', alpha=0.7, label='Novel')

# Scatter plot for R-Familiar group
plt.scatter([x_positions[2]] * len(Rfamiliar_familiar), 
            Rfamiliar_familiar, s=100, color='black', alpha=0.7, label='Rfamiliar')

# Add a dashed line at the mean value for Familiar group across the entire plot
#plt.axhline(y=average_xcorr_familiar, color='grey', linestyle='--', linewidth=2, label='Mean Familiar')

# Add labels and title
plt.ylabel('ΔF/F increased from familiar (%)', size=21)
plt.title('10th  minute', fontsize=21, pad=10)

# Set y-limits to make the plot cleaner
plt.ylim(-3.2, 9)
plt.xlim(0.97,1.05)

# Set custom x-ticks for the two groups (Novel, Rfamiliar)
plt.xticks(x_positions, ['Familiar(Avg)','Novel', 'Rfamiliar'], fontsize=20)
plt.yticks([-2,0, 2, 4, 6, 8], fontsize=20)

# Move the legend outside the plot and remove border
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, frameon=False)

# Remove top and right spines (border)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')  # Change border color to black
    spine.set_linewidth(2)     # Increase border thickness

# Show plot
plt.grid(False)
plt.tight_layout()
plt.show()



# Correlation speed vs dff (w/o filter) in 0 sec gap

In [ ]:
from numpy.lib.stride_tricks import as_strided


def _check_arg(x, xname):
    x = np.asarray(x)
    if x.ndim != 1:
        raise ValueError('%s must be one-dimensional.' % xname)
    return x

def autocorrelation(x, maxlag):
    """
    Autocorrelation with a maximum number of lags.

    `x` must be a one-dimensional numpy array.

    This computes the same result as
        numpy.correlate(x, x, mode='full')[len(x)-1:len(x)+maxlag]

    The return value has length maxlag + 1.
    """
    x = _check_arg(x, 'x')
    p = np.pad(x.conj(), maxlag, mode='constant')
    T = as_strided(p[maxlag:], shape=(maxlag+1, len(x) + maxlag),
                   strides=(-p.strides[0], p.strides[0]))
    return T.dot(p[maxlag:].conj())


def crosscorrelation(x, y, maxlag):
    """
    Cross correlation with a maximum number of lags.

    `x` and `y` must be one-dimensional numpy arrays with the same length.

    This computes the same result as
        numpy.correlate(x, y, mode='full')[len(a)-maxlag-1:len(a)+maxlag]

    The return vaue has length 2*maxlag + 1.
    """
    x = _check_arg(x, 'x')
    y = _check_arg(y, 'y')
    py = np.pad(y.conj(), 2*maxlag, mode='constant')
    T = as_strided(py[2*maxlag:], shape=(2*maxlag+1, len(y) + 2*maxlag),
                   strides=(-py.strides[0], py.strides[0]))
    px = np.pad(x, maxlag, mode='constant')
    return T.dot(px)

In [ ]:
xcorr_familiar=[]

for i in range(len(dff_f_mean)):    
    
        try:
        # Time series data
            switch1=switch[i].switch1.iloc[0]
            switch2=switch[i].switch2.iloc[0]
            data1 = dff_f_mean[i][:switch1]
            velocity_f = savgol_filter(df_downsample[i].Velocity,50,3)
            velocity_f=pd.DataFrame(velocity_f)
            data2=velocity_f[0][:switch1]
        # Detrend the data
            detrended_data1 = detrend(data1)
            detrended_data2 = detrend(data2)
        # Calculate cross-correlation
            cross_corr = crosscorrelation(detrended_data1, detrended_data2, 0) #0sec lag
        # Normalize the cross-correlation to range from -1.0 to 1.0
            normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
            xcorr_familiar.append(normalized_corr[0])
        except:
            print(i)
            print('ERROR!!!')
xcorr_familiar

In [ ]:
xcorr_min_novel = []

for i in range(len(dff_f_mean)):  
    switch1 = switch[i].switch1.iloc[0]
    switch2 = switch[i].switch2.iloc[0]
    subtracted_time = df_downsample[i].Time[switch1:switch2] - df_downsample[i].Time[switch1]
    velocity_f = savgol_filter(df_downsample[i].Velocity, 50, 3)
    velocity_f = pd.DataFrame(velocity_f)
    xcorr_min_novel_each = []
    
    for n in range(1,20+1):
        if n == 1:
            abs_diff = np.abs(subtracted_time - n * 60)
            X = np.argmin(abs_diff)
            data1 = dff_f_mean[i][switch1:switch2][:X]
            data2 = velocity_f[0][switch1:switch2][:X]
        else:
            abs_diff1 = np.abs(subtracted_time - (n - 1) * 60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(subtracted_time - n * 60)
            X2 = np.argmin(abs_diff2)
            data1 = dff_f_mean[i][switch1:switch2][X1:X2]
            data2 = velocity_f[0][switch1:switch2][X1:X2]
        
        # Detrend the data each time
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 0) # 0 sec lag
        
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        xcorr_min_novel_each.append(normalized_corr[0])
    
    xcorr_min_novel.append(xcorr_min_novel_each)
    
xcorr_min_novel

In [ ]:
xcorr_min_Rfamiliar = []

for i in range(len(dff_f_mean)):  
    switch1 = switch[i].switch1.iloc[0]
    switch2 = switch[i].switch2.iloc[0]
    subtracted_time = df_downsample[i].Time[switch2:] - df_downsample[i].Time[switch2]
    velocity_f = savgol_filter(df_downsample[i].Velocity, 50, 3)
    velocity_f = pd.DataFrame(velocity_f)
    xcorr_min_Rfamiliar_each = []
    
    for n in range(1,10+1):
        if n == 1:
            abs_diff = np.abs(subtracted_time - n * 60)
            X = np.argmin(abs_diff)
            data1 = dff_f_mean[i][switch2:][:X]
            data2 = velocity_f[0][switch2:][:X]
        else:
            abs_diff1 = np.abs(subtracted_time - (n - 1) * 60)
            X1 = np.argmin(abs_diff1)
            abs_diff2 = np.abs(subtracted_time - n * 60)
            X2 = np.argmin(abs_diff2)
            data1 = dff_f_mean[i][switch2:][X1:X2]
            data2 = velocity_f[0][switch2:][X1:X2]
        
        # Detrend the data each time
        detrended_data1 = detrend(data1)
        detrended_data2 = detrend(data2)
        
        # Calculate cross-correlation
        cross_corr = crosscorrelation(detrended_data1, detrended_data2, 0) # 0 sec lag
        
        # Normalize the cross-correlation to range from -1.0 to 1.0
        normalized_corr = cross_corr / np.sqrt(np.sum(detrended_data1 ** 2) * np.sum(detrended_data2 ** 2))
        xcorr_min_Rfamiliar_each.append(normalized_corr[0])
    
    xcorr_min_Rfamiliar.append(xcorr_min_Rfamiliar_each)
    
xcorr_min_Rfamiliar

In [ ]:
#Plot R^2 over time

# Legend labels
legend_labels = ["novel", "returned familiar"]

# Prepare data
data_familiar=xcorr_familiar
data_novel=np.array(xcorr_min_novel)
data_Rfamiliar=np.array(xcorr_min_Rfamiliar)

mean_familiar = np.mean(xcorr_familiar)
mean_novel = []
mean_Rfamiliar = []
ste_novel = []
ste_Rfamiliar = []

# Assuming each increased_* is a list of lists, we need to iterate through each sublist
for i in range(8):
    mean_Rfamiliar.append(np.mean(data_Rfamiliar[:,i]))
    ste_Rfamiliar.append(np.std(data_Rfamiliar[:,i]) / np.sqrt(len(data_Rfamiliar[:,i])))
for i in range(20):    
    mean_novel.append(np.mean(data_novel[:,i]))
    ste_novel.append(np.std(data_novel[:,i]) / np.sqrt(len(data_novel[:,i])))

# Combine the filtered data lists for easier iteration
mean_data_lists = [mean_novel, mean_Rfamiliar]
ste_data_lists = [ste_novel, ste_Rfamiliar]

# Define colors for each criterion
colors = ["red", "black"]

# Define lighter colors for filling
lighter_colors = ["#FFB6C1", "#A9A9A9"]

# Prepare figure
plt.figure(figsize=(6, 6))

# Iterate over each data list to plot
for i, (avg_data, std_error) in enumerate(zip(mean_data_lists, ste_data_lists)):
    # Convert mean and std error lists to NumPy arrays
    avg_data = np.array(avg_data)
    std_error = np.array(std_error)

    # Define x-values specific to the length of avg_data
    x_values = np.arange(1, len(avg_data) + 1)

    # Plot average data
    plt.plot(x_values, avg_data, label=legend_labels[i], color=colors[i], linewidth=2.5)

    # Plot standard error as filled area
    plt.fill_between(x_values, avg_data - std_error, avg_data + std_error, color=lighter_colors[i], alpha=0.5)

# Plot familiar_mean as a dashed line (without error bars)
familiar_mean_avg = np.mean(mean_familiar)
plt.plot(range(1, len(mean_data_lists[0]) + 1), [familiar_mean_avg] * len(mean_data_lists[0]), 
         label="familiar_mean", color="grey", linestyle="--", linewidth=4)

# Adding titles and labels
plt.title('Correlation to running speed', fontsize=22)
plt.xlabel('Time (min)', fontsize=22)
plt.ylabel('$R^2$', fontsize=22)

#plt.ylim(-0.2,-0.8)

# Set x-ticks
plt.xticks(ticks=[0, 5, 10, 15, 20], fontsize=18)
plt.yticks([0.3,0.4,0.5,0.6,0.7], fontsize=18)

# Change the thickness of x-ticks and y-ticks
plt.tick_params(axis='both', width=2)

# Adding a legend outside the plot
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=18)

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')
    spine.set_linewidth(2)

# Layout adjustment to accommodate legend
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
# Wilocxon

from scipy.stats import wilcoxon

data_familiar=xcorr_familiar
data_novel=np.array(xcorr_min_novel)
data_Rfamiliar=np.array(xcorr_min_Rfamiliar)

# Perform Wilcoxon signed-rank test for each time point
p_values_novel = []
p_values_Rfamiliar = []
p_values_Rnovel =[]

for i in range(10):
    # Compare familiar vs novel
    stat, p = wilcoxon(data_familiar, data_novel[:, i])
    p_values_novel.append(p)
for i in range(8):
    # Compare familiar vs returned familiar
    stat, p = wilcoxon(data_familiar,data_Rfamiliar[:, i])
    p_values_Rfamiliar.append(p)
    # Compare novel vs returned familiar
    stat, p = wilcoxon(data_novel[:,i],data_Rfamiliar[:, i])
    p_values_Rnovel.append(p)

# Print the p-values
print("P-values for familiar vs novel at each time point:", p_values_novel)
print("P-values for familiar vs returned familiar at each time point:", p_values_Rfamiliar)
print("P-values for novel vs returned familiar at each time point:", p_values_Rnovel)

In [ ]:
#Plot R^2 at 1st min

# Prepare data: Use the first 10 columns of each dataset
data_familiar=xcorr_familiar
data_novel=np.array(xcorr_min_novel)[:,0]
data_Rfamiliar=np.array(xcorr_min_Rfamiliar)[:,0]

# Calculate mean values
average_familiar = np.mean(data_familiar)
average_novel = np.mean(data_novel)
average_Rfamiliar = np.mean(data_Rfamiliar)

# Prepare data for bar plot
group_labels = ['Familiar','Novel', 'Rfamiliar']
mean_values = [average_familiar,average_novel, average_Rfamiliar]

# Adjust x-positions to give more space between the groups
x_positions = [0.98, 1.01, 1.04]  # Reduce the distance between the two bars

# Plot the bar plot, scatter plots, and dashed line
plt.figure(figsize=(6.6, 6.6))
for i in range(len(data_novel)):
    plt.plot([x_positions[0], x_positions[1]], 
             [data_familiar[i], data_novel[i]], 
             color='grey', alpha=0.5, linewidth=2)
    plt.plot([x_positions[1], x_positions[2]], 
             [data_novel[i], data_Rfamiliar[i]], 
             color='grey', alpha=0.5, linewidth=2)

# Scatter plot for familiar group
plt.scatter([x_positions[0]] * len(data_familiar), 
            data_familiar, s=100, color='grey', alpha=0.7, label='Familiar')    
    
# Scatter plot for Novel group
plt.scatter([x_positions[1]] * len(data_novel), 
            data_novel, s=100, color='red', alpha=0.7, label='Novel')

# Scatter plot for R-Familiar group
plt.scatter([x_positions[2]] * len(data_Rfamiliar), 
            data_Rfamiliar, s=100, color='black', alpha=0.7, label='Rfamiliar')

# Add a dashed line at the mean value for Familiar group across the entire plot
#plt.axhline(y=average_xcorr_familiar, color='grey', linestyle='--', linewidth=2, label='Mean Familiar')

# Add labels and title
plt.ylabel('$R^2$', size=21)
plt.title('1st minute', fontsize=21, pad=20)

# Set y-limits to make the plot cleaner
plt.ylim(-0.1, 0.9)
plt.xlim(0.97,1.05)

# Set custom x-ticks for the two groups (Novel, Rfamiliar)
plt.xticks(x_positions, ['Familiar(Avg)','  Novel', 'Rfamiliar'], fontsize=18)
plt.yticks([0,0.2,0.4,0.6,0.8,1.0], fontsize=18)

# Move the legend outside the plot and remove border
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, frameon=False)

# Remove top and right spines (border)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')  # Change border color to black
    spine.set_linewidth(2)     # Increase border thickness

# Show plot
plt.grid(False)
plt.tight_layout()
plt.show()


In [ ]:
#Plot R^2 at 5th min

# Prepare data: Use the first 10 columns of each dataset
data_familiar=xcorr_familiar
data_novel=np.array(xcorr_min_novel)[:,4]
data_Rfamiliar=np.array(xcorr_min_Rfamiliar)[:,4]

# Calculate mean values
average_familiar = np.mean(data_familiar)
average_novel = np.mean(data_novel)
average_Rfamiliar = np.mean(data_Rfamiliar)

# Prepare data for bar plot
group_labels = ['Familiar','Novel', 'Rfamiliar']
mean_values = [average_familiar,average_novel, average_Rfamiliar]

# Adjust x-positions to give more space between the groups
x_positions = [0.98, 1.01, 1.04]  # Reduce the distance between the two bars

# Plot the bar plot, scatter plots, and dashed line
plt.figure(figsize=(6.6, 6.6))
for i in range(len(data_novel)):
    plt.plot([x_positions[0], x_positions[1]], 
             [data_familiar[i], data_novel[i]], 
             color='grey', alpha=0.5, linewidth=2)
    plt.plot([x_positions[1], x_positions[2]], 
             [data_novel[i], data_Rfamiliar[i]], 
             color='grey', alpha=0.5, linewidth=2)

# Scatter plot for familiar group
plt.scatter([x_positions[0]] * len(data_familiar), 
            data_familiar, s=100, color='grey', alpha=0.7, label='Familiar')    
    
# Scatter plot for Novel group
plt.scatter([x_positions[1]] * len(data_novel), 
            data_novel, s=100, color='red', alpha=0.7, label='Novel')

# Scatter plot for R-Familiar group
plt.scatter([x_positions[2]] * len(data_Rfamiliar), 
            data_Rfamiliar, s=100, color='black', alpha=0.7, label='Rfamiliar')

# Add a dashed line at the mean value for Familiar group across the entire plot
#plt.axhline(y=average_xcorr_familiar, color='grey', linestyle='--', linewidth=2, label='Mean Familiar')

# Add labels and title
plt.ylabel('$R^2$', size=21)
plt.title('5th minute', fontsize=21, pad=20)

# Set y-limits to make the plot cleaner
plt.ylim(-0.1, 1)
plt.xlim(0.97,1.05)

# Set custom x-ticks for the two groups (Novel, Rfamiliar)
plt.xticks(x_positions, ['Familiar(Avg)','  Novel', 'Rfamiliar'], fontsize=18)
plt.yticks([0,0.2,0.4,0.6,0.8,1.0], fontsize=18)

# Move the legend outside the plot and remove border
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, frameon=False)

# Remove top and right spines (border)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')  # Change border color to black
    spine.set_linewidth(2)     # Increase border thickness

# Show plot
plt.grid(False)
plt.tight_layout()
plt.show()


In [ ]:
#Plot R^2 at 10th min

# Prepare data: Use the first 10 columns of each dataset
data_familiar=xcorr_familiar
data_novel=np.array(xcorr_min_novel)[:,9]
data_Rfamiliar=np.array(xcorr_min_Rfamiliar)[:,9]

# Calculate mean values
average_familiar = np.mean(data_familiar)
average_novel = np.mean(data_novel)
average_Rfamiliar = np.mean(data_Rfamiliar)

# Prepare data for bar plot
group_labels = ['Familiar','Novel', 'Rfamiliar']
mean_values = [average_familiar,average_novel, average_Rfamiliar]

# Adjust x-positions to give more space between the groups
x_positions = [0.98, 1.01, 1.04]  # Reduce the distance between the two bars

# Plot the bar plot, scatter plots, and dashed line
plt.figure(figsize=(6.6, 6.6))
for i in range(len(data_novel)):
    plt.plot([x_positions[0], x_positions[1]], 
             [data_familiar[i], data_novel[i]], 
             color='grey', alpha=0.5, linewidth=2)
    plt.plot([x_positions[1], x_positions[2]], 
             [data_novel[i], data_Rfamiliar[i]], 
             color='grey', alpha=0.5, linewidth=2)

# Scatter plot for familiar group
plt.scatter([x_positions[0]] * len(data_familiar), 
            data_familiar, s=100, color='grey', alpha=0.7, label='Familiar')    
    
# Scatter plot for Novel group
plt.scatter([x_positions[1]] * len(data_novel), 
            data_novel, s=100, color='red', alpha=0.7, label='Novel')

# Scatter plot for R-Familiar group
plt.scatter([x_positions[2]] * len(data_Rfamiliar), 
            data_Rfamiliar, s=100, color='black', alpha=0.7, label='Rfamiliar')

# Add a dashed line at the mean value for Familiar group across the entire plot
#plt.axhline(y=average_xcorr_familiar, color='grey', linestyle='--', linewidth=2, label='Mean Familiar')

# Add labels and title
plt.ylabel('$R^2$', size=21)
plt.title('10th minute', fontsize=21, pad=20)

# Set y-limits to make the plot cleaner
plt.ylim(-0.1, 1)
plt.xlim(0.97,1.05)

# Set custom x-ticks for the two groups (Novel, Rfamiliar)
plt.xticks(x_positions, ['Familiar(Avg)','  Novel', 'Rfamiliar'], fontsize=18)
plt.yticks([0,0.2,0.4,0.6,0.8,1.0], fontsize=18)

# Move the legend outside the plot and remove border
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14, frameon=False)

# Remove top and right spines (border)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Change border color and thickness
for spine in plt.gca().spines.values():
    spine.set_color('black')  # Change border color to black
    spine.set_linewidth(2)     # Increase border thickness

# Show plot
plt.grid(False)
plt.tight_layout()
plt.show()
